In [1]:
import tinny
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data_train = np.array(pd.read_csv('./data/classification/train.csv')).T
data_test = np.array(pd.read_csv('./data/classification/test.csv')).T

m, n = data_train.shape

X_train = data_train[1:n]
y_train = data_train[0]

X_train, X_test, y_train, y_test = train_test_split(X_train.T, y_train, train_size=35000, random_state=42)

X_train = X_train.T / 255.0
X_test = X_test.T / 255.0

In [3]:
print(y_train[:5])

[3 1 0 8 0]


In [4]:
l1 = tinny.DenseLayer(784, 10, "ReLU")
l2 = tinny.OutputLayer(10, 10, "softmax")
nn = tinny.TiNNyNetwork(problem_type="classification", loss_function="crossentropy", layers=[l1, l2])

nn.train(X_train, y_train, iterations=350, learning_rate=0.01)
nn.test(X_test, y_test)

Iteration 0
Loss: 0.8355691488908573
Accuracy: 0.09831428571428572


Iteration 10
Loss: 4.103125776530838
Accuracy: 0.3031714285714286
Iteration 20
Loss: 3.4216616087823013
Accuracy: 0.495
Iteration 30
Loss: 2.855612598558572
Accuracy: 0.6291428571428571
Iteration 40
Loss: 2.4574065952731887
Accuracy: 0.6381714285714286
Iteration 50
Loss: 2.9630703793386983
Accuracy: 0.7057428571428571
Iteration 60
Loss: 2.7609542733164694
Accuracy: 0.7396857142857143
Iteration 70
Loss: 2.582316038521627
Accuracy: 0.7667428571428572
Iteration 80
Loss: 2.4909552000694726
Accuracy: 0.7846857142857143
Iteration 90
Loss: 2.428093846235178
Accuracy: 0.7995428571428571
Iteration 100
Loss: 2.376807726720783
Accuracy: 0.8117714285714286
Iteration 110
Loss: 2.3418453191152606
Accuracy: 0.8218857142857143
Iteration 120
Loss: 2.3342339748339853
Accuracy: 0.8297142857142857
Iteration 130
Loss: 2.3166939936898086
Accuracy: 0.8363428571428572
Iteration 140
Loss: 2.3150118742506773
Accuracy: 0.8406285714285714
Iteration 150
Loss: 2.3119039495687064
Accuracy: 0.8456571428571429
Iterati